In [30]:
import numpy as np

In [29]:
dimensions = {
    '7300.2': 6,
    '7100.2': 5,
    '7400.2': 1,
    '7600.2': 2
}

coordinates = {
    "X": 0,
    "Y": 1,
    "Z": 2
}

In [135]:
with open("/home/kokokos/TFM/PH+/DM/mrci-PHM-001.out") as f:
    lines = f.readlines()
istart, istop = [], []
i,j = 0, 0
check = False

nstates = 14

dm_tot_comp = np.zeros((nstates, nstates,3))
dm_tot = np.zeros((nstates, nstates))

for iline, line in enumerate(lines):
    #istart = 0
    if "Transition moment calculation" in line:
        #istart[i] = iline
        istart.append(iline)
        #i += 1
        check = True
        #istop = 0
    if '**********************************************************************************************************************************' in line and check:
        istop.append(iline)
        #istop[j] = iline
        #j += 1
        check = False
    #if (istart != 0 and istop != 0):
        #print(istart, istop)
        #print(lines[istart:istop],istart,istop)
        #dm = extract_matrix_elements(lines[istart:istop])
        #print(dm)
        #istart=0

#print(istart,istop,len(istart),len(istop))

for i in range(len(istart)):
    dm,file2,file1 = extract_matrix_elements(lines[istart[i]:istop[i]])
    #print(dm)
    print(i,istart[i],istop[i],file1,file2)
    if file1 == "7300.2":
        if file2 == file1:
            dm_tot_comp[:6,:6,:] = dm
        elif file2 == "7100.2":
            dm_tot_comp[:6,6:11,:] = dm
    elif file1 == "7100.2":
        if file2 == file1:
            dm_tot_comp[6:11,6:11,:] = dm
        elif file2 == "7400.2":
            dm_tot_comp[6:11,:6,:] = dm
    elif file1 == "7400.2":
        if file2 == file1:
            dm_tot_comp[11,11,:] = dm
        elif file2 == "7600.2":
            dm_tot_comp[11,12:,:] = dm
    elif file1 == "7600.2":
        if file2 == file1:
            dm_tot_comp[12:,12:,:] = dm
        elif file2 == "7400.2":
            dm_tot_comp[12:,11:11,:] = dm

for i in range(nstates):
    for j in range(nstates):
        dm_tot[i,j] = np.sqrt((dm_tot_comp[i,j,0])**2 + (dm_tot_comp[i,j,1])**2 + (dm_tot_comp[i,j,2])**2)
        

6 6
0 2650 2743 7300.2 7300.2
5 6
1 2889 2928 7300.2 7100.2
6 5
2 3038 3077 7100.2 7300.2
5 5
3 3187 3256 7100.2 7100.2
1 1
4 3366 3379 7400.2 7400.2
2 1
5 3405 3416 7400.2 7600.2
1 2
6 3442 3454 7600.2 7400.2
2 2
7 3480 3501 7600.2 7600.2


In [130]:
def extract_matrix_elements(lines):
    file1 = lines[4].split()[5]
    file2 = lines[5].split()[5]
    dim_ket = dimensions[file1]
    dim_bra = dimensions[file2]
    dm = np.zeros((dim_bra, dim_ket, 3))
    print(dim_ket,dim_bra)
    for line in lines[6:]:
        if '!MRCI expec' in line or '!MRCI trans' in line:
            #print(line)
            splited = line.split()
            if "L" in splited[2]:
                continue
            dims = splited[2]
            dim1, dim2, coor = int(dims[1]) - 1, int(dims[9]) - 1, coordinates[dims[7]]
           # print(file1, file2, dim_ket, dim_bra, dim1, dim2, coor)
            
            dm[dim1, dim2, coor] = float(splited[3])

            #print(dm)
    return dm,file1,file2

In [36]:
extract_matrix_elements(lines[3480:3501])

7600.2 2 2 0 0 2 -0.10182620001
7600.2 2 2 0 1 1 -0.524553996387
7600.2 2 2 1 0 1 -0.524553996387
7600.2 2 2 1 1 2 0.295308659711
